<a href="https://colab.research.google.com/github/algo21-221040114/PytorchTest/blob/main/Model_Transformer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
pip install sweetviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 22.6 MB 22.4 MB/s 0:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 5.3 MB/s eta 0:00:00
  Created wheel for ydata-profiling: filename=ydata_profiling-0.0.dev0-py2.py3-none-any.whl size=344995 sha256=5d63a52f5d296ac84f41d0f1a8cc40fc8c1b36cc015e166a47f13dde8e0145e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpypgeva/wheels/43/c8/f4/c0ebc32d7f20fe89d0e92d90eaeef5f0c0594a89b6bc16b352
Successfully built ydata-profiling
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninsta

In [121]:
import pandas as pd
import numpy as np
import sweetviz as sv
from pandas_profiling import ProfileReport

import math
import torch
import torch.nn as nn

import seaborn as sns
import matplotlib.pyplot as plt

In [122]:
df = pd.read_csv('/content/sample_data/000001.SS.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-16,3107.669922,3177.790039,3023.300049,3170.709961,3170.709961,444700
2022-03-17,3215.010010,3260.169922,3202.929932,3215.040039,3215.040039,420800
2022-03-18,3207.149902,3260.780029,3197.360107,3251.070068,3251.070068,373000
2022-03-21,3255.620117,3267.500000,3223.389893,3253.689941,3253.689941,354200
2022-03-22,3249.540039,3279.110107,3239.530029,3259.860107,3259.860107,361200


In [ ]:
# Plot
fig, ax = plt.subplots()
ax.plot(df.index, df['High'], color='y', label='high')
ax.plot(df.index, df['Close'], color='b', label='close')
ax.set_title('000001.SS')
ax.legend()
plt.show()

In [ ]:
# EDA 
def eda_sweetviz(df, report_name, target=None):
    # target certain data
    sweet_report = sv.analyze(df, target_feat=target)
    # in notebook, download the file and open in the browser
    sweet_report.show_html(f'{report_name}.html')

def eda_sweetviz_compare(df1, df2, report_name, feature, target):
    # compare two datasets
    feature_config = sv.FeatureConfig(force_text=feature, force_cat=feature)
    sweet_report = sv.compare(df1, df2, feat_cfg=feature_config, target_feat=target)
    sweet_report.show_html(f'{report_name}_compare.html')


eda_sweetviz(df, 'Daily Data of 000001.SS')

def eda_pandas_profilling(df, report_name):
    # directly show in notebook
    profile = ProfileReport(df, title=report_name, explorative=True)
    return profile

eda_pandas_profilling(df, '000001.SS Report')

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [123]:
# Data Preprocess
cl = df['Close']
features = np.array((df['Open'], df['High'], df['Low'], df['Close'])).T
length = features.shape[0] # total time length
src = []
tgt = []
tgt_y = []

for i in range(length-12):
  s = features[i: i+10, :]
  src.append(s)

  t = features[i+11, :]
  tgt.append(t)

  t_y = features[i+12, -1] # close price
  tgt_y.append(t_y)

train_src = src[:200]
test_src = src[200:]
train_tgt = tgt[:200]
test_tgt = tgt[200:]
train_tgt_y = tgt_y[:200]
test_tgt_y = tgt_y[200:]

print(len(train_tgt_y))

200


In [124]:
def shuffle_data(feature, target, target_y):
    """
    :param feature: encoder输入特征, list
    :param target: decoder输入特征
    :param target_y: ground truth
    :param shuffle: 是否打乱顺序
    :return:
    """
    # 输入all_datas的每一项必须是numpy数组
    feature = [np.array(x) for x in feature]
    target = [np.array(x) for x in target]
    target_y = [np.array(x) for x in target_y]

    # 获取样本大小
    data_size = len(feature)

    # 随机生成打乱的索引
    p = np.random.permutation(data_size)
    # 重新组织数据
    feature = torch.Tensor(np.array(feature)[p])
    target =  torch.Tensor(np.array(target)[p]).unsqueeze(1)
    target_y =  torch.Tensor(np.array(target_y)[p])

    return feature, target, target_y

In [138]:
# 策略：使用前十个交易日的close信息预测当日close
class QuantModel(nn.Module):

  def __init__(self):
      super(QuantModel, self).__init__()

      self.transformer = nn.Transformer(d_model = 4,
                                        nhead=2,
                                        num_encoder_layers=12,
                                        num_decoder_layers=12,
                                        dim_feedforward=128,
                                        dropout=0.2,
                                        activation='relu',
                                        custom_encoder=None,
                                        custom_decoder=None,
                                        batch_first = True)
      self.out_linear = nn.Linear(4, 1)

  def forward(self, src, tgt):

      tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[1])
      
      out = self.transformer(src, tgt, tgt_mask=tgt_mask)

      return out

model = QuantModel()

In [150]:
criteria = nn.MSELoss()
# def MSE (y, y_predicted):
#    sq_error = (y_predicted - y) ** 2
#    sum_sq_error = np.sum(sq_error)
#    mse = sum_sq_error/y.size
#    return mse
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [147]:
batch_size = 5
src1 = torch.rand((batch_size, 10, 4))
tgt1 = torch.rand((batch_size, 1, 4))
tgt_y1 = torch.rand((batch_size, 1, 1))
out = model(src1, tgt1)
output = model.out_linear(out)
print(output.size())
print(output.contiguous().view(-1, output.size(-1)).size())
print(tgt_y1.contiguous().view(-1, tgt_y1.size(-1)).size())
a = output.contiguous().view(-1, output.size(-1)).detach().numpy()
b = tgt_y1.contiguous().view(-1, tgt_y1.size(-1)).detach().numpy()
loss = MSE(a, b)
print(loss)

torch.Size([5, 1, 1])
torch.Size([5, 1])
torch.Size([5, 1])
0.40796051025390623


In [ ]:
src, tgt, tgt_y = shuffle_data(train_src, train_tgt, train_tgt_y)
print(src.size())
print(tgt.size())
print(tgt_y.size())

torch.Size([200, 10, 4])
torch.Size([200, 1, 4])
torch.Size([200])


In [152]:
# Train model
data_size = 200
batch_size = 10
total_loss = 0
epochs = 20
batch_count = data_size//batch_size

for epoch in range(1, epochs + 1):

    total_loss = 0
    optimizer.zero_grad()
    src, tgt, tgt_y = shuffle_data(train_src, train_tgt, train_tgt_y)

    for batch in range(batch_count+1):
      start = batch*batch_size
      if start >= data_size:
        break
      else:
        end = min((batch+1)*batch_size, data_size)
          
        src1 = src[start:end]
        tgt1 = tgt[start:end]
        tgt_y1 = tgt_y[start:end]
        out = model(src1, tgt1)
        out = model.out_linear(out)
        loss = criteria(out.contiguous().view(-1, out.size(-1)).detach().numpy(), 
                  tgt_y1.contiguous().view(-1, tgt_y1.size(-1)).detach().numpy())

        loss.backward()
        optimizer.step()
        total_loss += loss

    # 每40次打印一下loss
    if epoch != 0 and epoch % 10 == 0:
        print("Step {}, total_loss: {}".format(epoch, total_loss))
        total_loss = 0

TypeError: ignored

In [148]:
print(99//10)

9


In [ ]:
# Test model